In [1]:
import os
import json

import numpy as np
import pandas as pd

import tensorflow as tf

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import joblib

In [3]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [4]:
data_folder = '/home/developer/gcp/cbidmltsf/datasets/electricity'

In [5]:
# constant values for positional encodings
hours_in_day = 24
days_in_week = 7
days_in_month = 30
days_in_year = 365
# weeks_of_year and month_of_year become redundant when using days_of_year, do not evaluate them
# weeks_in_year = 52
# months_in_year = 12

In [6]:
# split the time series in seen (train, eval) and unseen (test) data
# according to academic papers:

# 243 days on seen data, 7 days on unseen data 

# seen data:      '2014-01-01 00:00:00' to '2014-08-31 23:00:00', 243*24 = 5832 lectures

# train/eval split is 0.9/0.1, then

# train data:     '2014-01-01 00:00:00' to '2014-08-07 15:00:00', 5248 lectures
# eval data:      '2014-08-07 15:00:00' to '2014-08-31 23:00:00', 584 lectures

# unseen data:    '2014-09-01 00:00:00' to '2014-09-07 23:00:00', 7*24 = 168 lectures

# 243 weeks for seen data, 1 week for unseen data
no_lectures_seen_data = 243*24 # 5832

# seen data is divided as 90% for training and 10% for evaluation
train_eval_limit = 0.9

train_interval_end = int(no_lectures_seen_data*train_eval_limit) # 5248

In [7]:
# build sub-series to be persisted as serialized training examples

# dimensionality of the encoder input
m = 168

# dimensionality of the decoder output 
t = 168

In [8]:
# columns to be included in the SLDB
sldb_columns = [
    'date',
    'token_id',
    'kw_scaled',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    'sin_day_month',
    'cos_day_month',
    'sin_day_year',
    'cos_day_year'    
]

In [9]:
sldb = {
    'ts': 'LD2011-2014_SEPARATED_MT_320-MT_330',
    'embedding': {
        'hourly': 168
    },
    'tau': {
        'hourly': 1
    },
    'no_targets': 168,
    'BSCTRFM': 1,
    'preprocessed': 1
}

In [10]:
sldb

{'ts': 'LD2011-2014_SEPARATED_MT_320-MT_330',
 'embedding': {'hourly': 168},
 'tau': {'hourly': 1},
 'no_targets': 168,
 'BSCTRFM': 1,
 'preprocessed': 1}

In [11]:
# a string with the basic specifications of the SLDB, as part of the SLDB identifier
sldb_specs = 'BSCTRFM_{:03d}_{:03d}'.format(sldb['embedding']['hourly'], sldb['no_targets'])
sldb_specs

'BSCTRFM_168_168'

In [12]:
# get the time-based identifier for the SLDB
sldb_identifier = '{}_{}'.format(sldb['ts'], sldb_specs)
sldb_identifier

'LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168'

In [13]:
sldb_dir = '/home/developer/gcp/cbidmltsf/sldbs/{}'.format(sldb_identifier)
sldb_dir

'/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168'

In [14]:
# make a directory for the complete SLDB
try:
    os.mkdir(sldb_dir)
    print('Directory {} was created.'.format(sldb_dir))
except FileExistsError:
    print('Error: directory {} already exists.'.format(sldb_dir))

Error: directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168 already exists.


In [15]:
# make a sub-directory for the training TFRecord files
try:
    os.mkdir('{}/train'.format(sldb_dir))
    print('Directory {}/train was created.'.format(sldb_dir))
except FileExistsError:
    print('Error: directory {}/train already exists.'.format(sldb_dir))

Error: directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168/train already exists.


In [16]:
# make a sub-directory for the evaluation TFRecord files
try:
    os.mkdir('{}/eval'.format(sldb_dir))
    print('Directory {}/eval was created.'.format(sldb_dir))
except FileExistsError:
    print('Error: directory {}/eval already exists.'.format(sldb_dir))

Error: directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168/eval already exists.


In [17]:
# make a sub-directory to persist time series pickles used for inference
try:
    os.mkdir('{}/test'.format(sldb_dir))
    print('Directory {}/test was created.'.format(sldb_dir))
except FileExistsError:
    print('Error: directory {}/test already exists.'.format(sldb_dir))

Directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168/test was created.


In [18]:
# get a path to the scalers sub-directory
scalers_dir = '{}/scalers'.format(sldb_dir)
scalers_dir

'/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168/scalers'

In [19]:
# make a sub-directory for the scalers
try:
    os.mkdir(scalers_dir)
    print('Directory {} was created.'.format(scalers_dir))
except FileExistsError:
    print('Error: directory {} already exists.'.format(scalers_dir))

Error: directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168/scalers already exists.


In [20]:
encoder_input_columns = [
    'kw_scaled',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    'sin_day_month',
    'cos_day_month',
    'sin_day_year',
    'cos_day_year'
]

In [21]:
# both the encoder input and the decoder input use the same columns from the source sub_series dataframe
decoder_input_columns = encoder_input_columns

In [22]:
target_columns = ['kw_scaled']

In [23]:
id_columns = ['token_id']

In [24]:
# a dictionary to manage data per individual customer_id
data = dict()

In [25]:
# a dictionary to manage a MinMaxScaler per individual customer_id
min_max = dict()

# a dictionary to manage a StandardScaler per individual customer_id
standard = dict()

In [26]:
# the number of cores available for training in Cloud TPU
num_cores = 8

In [27]:
start, end = 320, 330

In [28]:
token_ids = [token_id for token_id in np.arange(start, end + 1)]

In [29]:
customer_ids = ['MT_{:03d}'.format(token_id) for token_id in token_ids]

In [30]:
# are we training over raw data or preprocessed data?
state = 'preprocessed'

In [31]:
for customer_id in customer_ids:
    customer_data_path = '{}/separated_{}/{}.pkl'.format(data_folder, state, customer_id)
    data[customer_id] = pd.read_pickle(customer_data_path)

### for SLDB generation, run this unified code!

In [51]:
for token_id in token_ids:
    
    # initialize the examples dictionary for each customer
    examples = {
        'train': [],
        'eval': [],
        # test dataset is not passed to SLDB
        # 'test': []
    }

    # get the customer identifier
    customer_id = 'MT_{:03d}'.format(token_id)
    customer_id
    print('Started processing for {}'.format(customer_id))

    # a temporary dataframe with data per customer_id to build the sub-series/examples
    # data_df = filtered_output[filtered_output['token_id'] == token_id].copy()
    
    # use now a reference to the dataframe in the data dictionary 
    data_df = data[customer_id]

    # expand with positional encodings
    data_df['sin_hour_day'] = np.sin(2*np.pi*data_df.hour_of_day/hours_in_day)
    data_df['cos_hour_day'] = np.cos(2*np.pi*data_df.hour_of_day/hours_in_day)
    data_df['sin_day_week'] = np.sin(2*np.pi*data_df.day_of_week/days_in_week)
    data_df['cos_day_week'] = np.cos(2*np.pi*data_df.day_of_week/days_in_week)
    data_df['sin_day_month'] = np.sin(2*np.pi*data_df.day_of_month/days_in_month)
    data_df['cos_day_month'] = np.cos(2*np.pi*data_df.day_of_month/days_in_month)
    data_df['sin_day_year'] = np.sin(2*np.pi*data_df.day_of_year/days_in_year)
    data_df['cos_day_year'] = np.cos(2*np.pi*data_df.day_of_year/days_in_year)

    # get a series for the power usage variable on the training dataset, to fit the scaler
    lectures_train_data = data_df['power_usage'][:train_interval_end]

    # fit a scaler only on train data
    # it is required to pass the power usage time series to a (?, 1) NumPy array
    lectures_train_data_array = np.array(lectures_train_data).reshape(-1, 1)

    # get MinMaxScaler on train data, store it in a dictionary
    min_max_scaler = MinMaxScaler()
    min_max = min_max_scaler.fit(lectures_train_data_array)
    print('MinMax scaler generated on training data for {}'.format(customer_id))

    # persist the scaler
    scaler_filename = '{}/{}_min_max.save'.format(scalers_dir, customer_id)
    joblib.dump(min_max, scaler_filename)
    print('MinMax scaler persisted for {}'.format(customer_id))
    
    # get an array from the variable time series (seen and unseen)
    all_data_variable_array = np.array(data_df.power_usage).reshape(-1, 1)

    # apply the scaler over all data (seen and unseen)
    # rescale, and squeeze to drop the extra dimension, then assign to the new column kw_scaled
    data_df['kw_scaled'] = np.squeeze(min_max.transform(all_data_variable_array))
    
    # at this moment, the individual time series is ready to be window-rolled to produce
    # sub-series/examples to serialize
    
    # BSCTRFM inference process is not direct, but iterative, therefoer
    # no TFRecord SLDB is required for test dataset,
    
    # persist only the time series corresponding to the inference interval as test dataset
    data_to_persist = data_df[sldb_columns][no_lectures_seen_data - (m + t) + 1:]
    
    # path to persist the time series dataframe corresponding to test dataset
    path = '{}/test/{}.pkl'.format(sldb_dir, customer_id)
    
    data_to_persist.to_pickle(path)
    print('Test dataset persisted as a time series pickle for {}'.format(customer_id))
    
    
    # get an iterable with all the possible sub-series for training examples
    for starting_point in np.arange(train_interval_end - (m + t) + 1):

        sub_series_df = data_df[sldb_columns][starting_point:starting_point + (m + t)]

        encoder_input_df = sub_series_df[encoder_input_columns][:m]
        decoder_input_df = sub_series_df[decoder_input_columns][m-1:m-1+t]
        target_df = sub_series_df[target_columns][m:m+t]
        id_df = sub_series_df[id_columns][:1]

        encoder_input_list = encoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
        decoder_input_list = decoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
        target_list = target_df.reset_index(drop=True).to_numpy().flatten().tolist()
        id_list = id_df.reset_index(drop=True).to_numpy().flatten().tolist()

        examples['train'].append(
            {
                'encoder_input': encoder_input_list,
                'decoder_input': decoder_input_list,
                'target': target_list,
                'id': id_list,
            }
        )

    print('{} processed. The number of examples in {} dataset is {}'.\
          format(customer_id, 'train', len(examples['train'])))


    # ToDo: remove evaluation step from Cloud TPU training and use all seen data for training stage
    build_eval_set = True

    if build_eval_set:

        # get an iterable with all the possible sub-series for evaluation examples
        for starting_point in np.arange(train_interval_end, no_lectures_seen_data - (m + t) + 1):

            sub_series_df = data_df[sldb_columns][starting_point:starting_point + (m + t)]

            encoder_input_df = sub_series_df[encoder_input_columns][:m]
            decoder_input_df = sub_series_df[decoder_input_columns][m-1:m-1+t]
            target_df = sub_series_df[target_columns][m:m+t]
            id_df = sub_series_df[id_columns][:1]

            encoder_input_list = encoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
            decoder_input_list = decoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
            target_list = target_df.reset_index(drop=True).to_numpy().flatten().tolist()
            id_list = id_df.reset_index(drop=True).to_numpy().flatten().tolist()

            examples['eval'].append(
                {
                    'encoder_input': encoder_input_list,
                    'decoder_input': decoder_input_list,
                    'target': target_list,
                    'id': id_list,
                }
            )

        print('{} processed. The number of examples in {} dataset is {}'.\
              format(customer_id, 'eval', len(examples['eval'])))


    # DO NOT PRODUCE A TEST DATASET FOR SLDB, AS INFERENCE PROCESS IS NOT DIRECT
    # (IT IS ITERATIVE OVER UNSEEN DATA TIME SERIES)

    # on each customer dataset, adjust the number of examples to the number of training cores
    for stage in ['train', 'eval']:
        # how many examples/rows must be removed from examples[stage] to comply with the number of cores
        examples_to_remove = len(examples[stage])%num_cores

        # remove the last 'examples_to_remove' examples from the dataset
        for _ in np.arange(examples_to_remove):
            examples[stage].pop(-1)

        print('For {} cores in Cloud TPU, the number of {} examples for {} was adjusted to {}'.\
             format(num_cores, stage, customer_id, len(examples[stage])))


    # serialize the rows in examples['train'] and, if present, examples['eval']
    # to avoid excesive memory consumption

    # write a TFRecord file for each consumer_id/stage
    for stage in ['train', 'eval']:
        # N_ROWS = sldb['stats'][stage]['n_rows']
        N_ROWS = len(examples[stage])
        filename = '{}/{}/{}.tfrecord'.format(sldb_dir, stage, customer_id)

        with tf.io.TFRecordWriter(filename) as writer:
            for row in np.arange(N_ROWS):

                example = tf.train.Example(
                    # features within the example
                    features=tf.train.Features(
                        # individual feature definition
                        feature={'encoder_input':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['encoder_input']),
                                 'decoder_input':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['decoder_input']),
                                 'target':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['target']),
                                 'id':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['id'])
                                 }
                    )
                )
                serialized_example = example.SerializeToString()
                writer.write(serialized_example)

            # report TFRecord file as completed
            print('Persisted {} TFRecord file for {}'.format(stage, customer_id))

Started processing for MT_320
Test dataset persisted as a time series pickle for MT_320
Started processing for MT_321
Test dataset persisted as a time series pickle for MT_321
Started processing for MT_322
Test dataset persisted as a time series pickle for MT_322
Started processing for MT_323
Test dataset persisted as a time series pickle for MT_323
Started processing for MT_324
Test dataset persisted as a time series pickle for MT_324
Started processing for MT_325
Test dataset persisted as a time series pickle for MT_325
Started processing for MT_326
Test dataset persisted as a time series pickle for MT_326
Started processing for MT_327
Test dataset persisted as a time series pickle for MT_327
Started processing for MT_328
Test dataset persisted as a time series pickle for MT_328
Started processing for MT_329
Test dataset persisted as a time series pickle for MT_329
Started processing for MT_330
Test dataset persisted as a time series pickle for MT_330


In [56]:
# expand the sldb dictionary with final statistics

In [61]:
sldb['stats'] = {
    'train': {
        'n_rows': 54032,
    },
    'eval': {
        'n_rows': 2728
    }
}

In [62]:
sldb

{'ts': 'LD2011-2014_SEPARATED_MT_320-MT_330',
 'embedding': {'hourly': 168},
 'tau': {'hourly': 1},
 'no_targets': 168,
 'BSCTRFM': 1,
 'stats': {'train': {'n_rows': 54032}, 'eval': {'n_rows': 2728}}}

In [63]:
# serialize the sldb from the examples dictionary (keys are stages, values are lists of rows)

In [64]:
json_filename = '{}/sldb.json'.format(sldb_dir)

In [65]:
with open(json_filename, 'w') as filename:
    json.dump(sldb, filename, indent=4)

### ToDo: read TFRecord file into a Dataset and confirm the values in the source dataframe!!!

In [67]:
# do not forget to sync sldbs/ from local to GS after the previous operations!
!gsutil rsync -d -r /home/developer/gcp/cbidmltsf/sldbs gs://cbidmltsf/sldbs

Building synchronization state...
Starting synchronization...
Copying file:///home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168/eval/MT_320.tfrecord [Content-Type=application/octet-stream]...
Copying file:///home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168/eval/MT_321.tfrecord [Content-Type=application/octet-stream]...
Copying file:///home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168/eval/MT_322.tfrecord [Content-Type=application/octet-stream]...
Copying file:///home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168/eval/MT_323.tfrecord [Content-Type=application/octet-stream]...
\ [4 files][ 12.2 MiB/ 12.2 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rsync ... Please
see the -m section under "gsutil help options" f